<H1 > <CENTER>PROJET DE WEB SCRAPING</CENTER> </H1>
<i>Ce projet vise à évaluer les acquis des apprenants après la formation de Web Scraping. Il s’agira de collecter les informations sur les loyers dans la ville d’Abidjan sur les 10 dernières années.Chaque groupe sera constitué de 2 personnes et se verra affecter une commune d’Abidjan (pour la collecte des données) selon la répartition ci-dessous.Le site où rechercher les informations est Facebook.com</I>

<H2> PARTIE A : PREPARATION DES BESOINS:</H2>

<H2> 1 -IMPORTATION DES PACKAGES NECESSAIRE  </H2>

In [447]:
import numpy as np
import pandas as pd
import time
from datetime import datetime
from selenium import webdriver 
from selenium.webdriver.common.by import By 

# Importation de selenium
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re

import dateparser

# ! pip install unidecode 
from unidecode import unidecode 
# from ..src.access import email

<H2> 2-COORDONNEES DE CONNECTION A FACEBOOOK  </H2>

In [448]:


# password = access.password
# email = access.email

username_input = '//*[@id="email"]'
password_input = '//*[@id="pass"]'
login_submit = '//*[@name = "login"]'

# password = access.password
# email = access.email


password = "IKN1997"
email = "0566805898"

url_mbadon = 'https://web.facebook.com/groups/1558562987708936/search/?q=mbadon&_rdc=10&_rdr'
url_ciad = 'https://web.facebook.com/groups/1558562987708936/search/?q=ciad'


<H2>3- DEFINITIONS DE FONCTIONS NECESSAIRES POUR NOS OPERATIONS </H2>

<CENTER>  LES FONCTIONS</CENTER> 

<hr>

<table>
  <tr>
    <th>Fonctions</th>
    <th>Parametres</th>
    <th>Utilités</th>
  </tr>
  <tr>
    <td>get_all_publications</td>
    <td>url :  une adresse url</td>
    <td>Ouvrir une session sur facebook et recuperer les publications dans l'url passé</td>
  </tr>
  <tr>
    <td>get_links</td>
    <td>publications: les publications obtenues</td>
    <td>Extraction de lien dans les publications</td>
  </tr>
  <tr>
    <td>get_title</td>
    <td>poublications</td>
    <td>Extraction de titres des publications</td>
  </tr>
  <tr>
    <td>get_price</td>
    <td>publications </td>
    <td>Extraction des prix des maisons dans les publications</td>
  </tr>
  <tr>
    <td>get_texte</td>
    <td>publications</td>
    <td>Extraction de texte specifique au informations de la maison dans la publications</td>
  </tr>
  <tr>
    <td>get_publication</td>
    <td>url, driver</td>
    <td>Extraction de donnees dans une url specifique, cette methode ne necessite de d'authentification, donc plus de possibilité de se faire bloquer le compte, on navigue sur les groupes</td>
  </tr>
  <tr>
    <td>get_date</td>
    <td>urls, l'ensemble des urls</td>
    <td>Extraire les dates dans ces url, les dates n'etaient pas contenues dans les pages precedentes</td>
  </tr>
  <tr>
    <td>transform_string_date</td>
    <td>str_val : une chaine de caractère (la date au format string)</td>
    <td>Convertir les dates au format String et des formats date</td>
  </tr>
  <tr>
    <td>estAppart</td>
    <td>title : le titre du commentaire, texte: le texte associé à la post</td>
    <td>Identifier le type de l'offre pour un post, appartement(immeuble) ou maison(maison individuel)</td>

  <tr>
    <td>AppartDetect</td>
    <td>taille: tailles des post,  list_texte: liste de texte de tous les post, list_title: liste de titre de tous les post</td>
    <td>Appliquer l'identification de type sur tous les posts</td>
  </tr>
  <tr>
    <td>correction_price</td>
    <td>price:la liste des prix de logement de chaque post</td>
    <td>Corriger les valeurs abberantes sur les prix, (ex : 80$ veut dire 80 000, apres controle approfondi)</td>
  </tr>

  <tr>
    <td>standing</td>
    <td>prix: liste de prix de tous les post</td>
    <td>Categoriser les logements selon le prix (3 categorie)</td>
  </tr>
  <tr>
    <td>nb_piece</td>
    <td>texte :  le texte lié a la publication</td>
    <td>Identifier le nombre de pièces pour un texte d'une publication</td>
  </tr>

  <tr>
    <td>get_room_numb</td>
    <td>liste_texte :  la liste des texte lié aux publications</td>
    <td>Identifier le nombre de pièces pour chaque texte</td>
  </tr>
  
</table>

</ol>

<LI> get_all_publications</LI> 

In [449]:
## Get all publications available on the page
def get_all_publications(url) : 
    driver = webdriver.Firefox()

# Ouvre la page Facebook
    driver.get("https://www.facebook.com/")
    driver.find_element(By.XPATH, username_input).send_keys(email)
    time.sleep(5)
    driver.find_element(By.XPATH, password_input).send_keys(password)
    time.sleep(3)
    driver.find_element(By.XPATH, login_submit).click()
    time.sleep(20)
    driver.get(url)
    time.sleep(20)
    publications = driver.find_elements(By.XPATH, "//div[@class='x9f619 x1n2onr6 x1ja2u2z x78zum5 xdt5ytf x193iq5w xeuugli x1r8uery x1iyjqo2 xs83m0k xsyo7zv x16hj40l x10b6aqq x1yrsyyn']")
    time.sleep(30)
    return publications

<LI> get_links</LI> 

In [450]:
def get_links(publications): 
    
    list_links = []
   
    for publication in publications:
        # Récupère tous les liens dans la publication
        links = publication.find_elements(By.XPATH, ".//a")
        # Ajoute chaque lien à la liste globale
        for link in links:
            href = link.get_attribute("href")
            list_links.append(href)

 

    return list_links

<LI> get_title</LI> 

In [451]:
def get_title(publications): 
    
    list_title = []
   
    for publication in publications:
        # Récupère tous les liens dans la publication
        titles = publication.find_elements(By.XPATH, ".//a")
        # Ajoute chaque lien à la liste globale
        for title in titles:
            title_name = title.text
            list_title.append(title_name)
    return list_title

<LI> get_price</LI> 

In [452]:

def get_price(publications): 
    
    list_price = []
   
    for publication in publications:
        # Récupère tous les liens dans la publication
        prices = publication.find_elements(By.XPATH, "//span[@class='x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x676frb x1nxh6w3 x1sibtaa xo1l8bm x6u5lvz x1yc453h']")
        # Ajoute chaque lien à la liste globale
        for price in prices:
            price_name= price.text
            list_price.append(price_name)
        break
    return list_price    

<LI> get_texte</LI> 

In [453]:


def get_texte(publications): 
    
    list_texte = []
   
    for publication in publications:

        # Récupère tous les liens dans la publication
        textes = publication.find_elements(By.XPATH,'//div[@class="x9f619 x1n2onr6 x1ja2u2z x78zum5 xdt5ytf x193iq5w xeuugli x1iyjqo2 xs83m0k x1l7klhg xsyo7zv x16hj40l x10b6aqq x1yrsyyn"]/span[@class ="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen xo1l8bm xzsf02u"]/span[@class ="x1lliihq x6ikm8r x10wlt62 x1n2onr6"]')
        # Ajoute chaque lien à la liste globale
        for texte in textes:
            texte_name= texte.text
            list_texte.append(texte_name)
        break
    return list_texte  



<LI> Sauvegarde</LI> 

In [454]:

# import pickle
# def Sauvegarde(var, var_name):
#     with open(var_name, 'wb') as f1:
#         pickle.dump(var, f1)
#     print("Sauvegarde terminée")

# def Chargement(var_name):
#     with open(var_name, 'rb') as f1:
#         OD = pickle.load(f1)
#     return OD

<LI> get_publication</LI> 

In [455]:

## Get all publications available on the page
def get_publication(url,driver) : 
    # driver = webdriver.Firefox()
    driver.get(url)
    list_texte = []
    time.sleep(10)
    publications = driver.find_elements(By.XPATH, "//a/span")
    
    for texte in publications:
        texte_name= texte.text
        driver.quit
        return texte_name
  

<LI> get_date</LI> 

In [456]:
def get_date(urls) :
    driver = webdriver.Firefox()
    liste = []
    for url in   urls:
        time.sleep(5)
        temp = get_publication(url,driver)
        print(temp)
        liste.append(temp)
        time.sleep(5)
    driver.quit
    return liste

<LI> transform_string_date</LI> 

In [507]:
##   Convert the string date format to datetime 


def transform_string_date(str_val):
    str_val = str_val.lower()
    datte = dateparser.parse(str_val)
    
    
    if 'instant' in str_val:
        datte = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
    else:
        datte.strftime("%Y-%m-%d %H:%M:%S")

    return str_val.date()

#####################

def string_to_date(date):
       
   
    date_list = []

    for dat in date:
        try:
            date_list.append(dateparser.parse(dat))
        except:
            date_list.append(None)


    return date_list


def date_to_year(date):
       
   
    date_list = []

    for dat in date:
        try:
            date_list.append(dat.year)
        except:
            date_list.append(None)


    return date_list

def date_to_month(date):
       
   
    date_list = []

    for dat in date:
        try:
            date_list.append(dat.month)
        except:
            date_list.append(None)


    return date_list

def date_to_day(date):
       
   
    date_list = []

    for dat in date:
        try:
            date_list.append(dat.day)
        except:
            date_list.append(None)


    return date_list

<LI> estAppart</LI> 

In [458]:
def estAppart(title, texte):
    dic = {True: "Appartement", False: "Maison"}
    compltexte = texte + ' '+ title
    compltexte = compltexte.split() 
    compltexte = [mot.upper() for mot in compltexte]

    motif1 = ['APPARTEMENT','APPARTEMENTS','APARTEMENT','APARTEMENTS',  'APPATEMENT','APPATEMENTS', 'APPAT', 'APPATS']
    motif2 = ['IMMEUBLE','IMMEUBLES','IMEUBLE','IMMEBLE',  'IMMEBLES','IMEBLE', 'IMEBLES'] 

    test1 = any([ i in compltexte for i in motif1])
    test2 = any([ i in compltexte for i in motif2])

    what = (test1 or test2)
    return dic[what]
    

<LI> AppartDetect</LI> 

In [459]:
def AppartDetect(taille,list_texte,list_title):
    type = []
    for i in range(taille):
        type.append(estAppart(list_texte[i], list_title[i]))
    return type

<LI> Correction_price</LI> 

In [460]:
def correction_price(price):
    try:
        price = [prx.replace('Gratuit','0') for prx in price]
        price = [prx.replace('€','') for prx in price]
        price = [prx.replace('$','') for prx in price]
        price = [prx.replace('FCFA','') for prx in price]
        price = [prx.replace(' ','') for prx in price]
        price = [int(prx) for prx in price]
    except:
        pass
    
    for i in range(len(price)):
        if i<1000:
            price[i] = price[i] * 1000
        else:
            price[i] = price[i]

    return price
    

<LI> Standing</LI> 

In [461]:
def standing(prix):
   categorie = np.percentile(list_ciad_price, [34,67]) 
   stand = []
   for price in prix:
      if price <= categorie[0]:
         stand.append("Economique")
      elif price > categorie[1]:
         stand.append("Haut")
      else: 
         stand.append("Moyen")
   return stand

<LI> nb_piece</LI> 

In [462]:

from unidecode import unidecode
import re
def nb_piece(texte):

    compltexte = texte

    compltexte = compltexte.replace("*", "")
    compltexte = compltexte.replace(",", "")
    compltexte = compltexte.replace(".", "")
    

    compltexte = compltexte.split() 
    compltexte = [mot.upper() for mot in compltexte]
    compltexte = [unidecode(mot) for mot in compltexte]
    nb_pieces = None


    motif = ["PIECE", "PIECES","PCS","PIECE,","PIEGE","PIEGES"]

    for i in range(1,len(compltexte)):
        try:
            if compltexte[i] in motif:
                # break
                nb_pieces = compltexte[i-1]
                nb_pieces = nb_pieces.replace(" ", "")
                nb_pieces = re.findall("\d+", nb_pieces)
                nb_pieces = int(nb_pieces[0])              
                return nb_pieces
                
        except:
                pass
    return nb_pieces


<LI> get_room_numb</LI> 

In [463]:

def get_room_numb(liste_texte):

    liste = []
    for texte in liste_texte:
      liste.append(nb_piece(texte))
    return liste

<H2> PARTIE B  : APPLICATION </H2>

<H2>1- CIAD</H2>

In [464]:
# OBTENTION DES PUBLICATION
publication_ciad = get_all_publications(url_ciad)

In [465]:
# OBTENTION DES LIENS
list_ciad_links = get_links(publication_ciad)
len(list_ciad_links)

34

In [466]:
# OBTENTION DES TITLES
list_ciad_title = get_title(publication_ciad)
len(list_ciad_title)
list_ciad_title[1:3]


['Location Appartement chic à la Riviera CIAD …',
 'MERVEILEUX APPARTS MEUBLÉS À LA CITÉ CIAD ( COCODY)']

In [467]:
# OBTENTION DES PRIX
list_ciad_price = get_price(publication_ciad)
print(len(list_ciad_price))
list_ciad_price[0:3]

34


['500 000 FCFA', '180 000 FCFA', '60 000 FCFA']

In [468]:
# OBTENTION DES TEXTES

list_ciad_texte = get_texte(publication_ciad)

print(len(list_ciad_texte))
list_ciad_texte[0:2]

34


["🍅 A LOUÉ JOLIE VILLA BON STANDING A LA RIVIERA 4 CIAD CITÉ EDEN 🍅 A louer villa basse 4 pièces + 1 dépendant à Rivera CIAD CITÉ EDEN avec 4 salles d'eaux, cours, jardin, garage LOYER : 500mille CONTACT :: 0707973469",
 "COCODY RIVIÉRA CIAD LOCATION D'UN JOLI APPARTEMENT 02 PIÈCES AU AU REZ AVEC UN GRAND SALON, 02 SALLES D'EAU ET UN PARKING INTÉRIEUR LOYER: 185.000F CONDITIONS: 05 MOIS ☎️…+225 0778237066(Whassap)"]

In [469]:
# CREATION DE VARIABLES "CONSTANTE"

list_ciad_com = ["Cocody/Riveira" for i  in range(len(list_ciad_texte))]
list_ciad_quart = ["Ciad" for i  in range(len(list_ciad_texte))]

len(list_ciad_com), len(list_ciad_quart)


(34, 34)

In [472]:
# OBTENTION DES DATES DES PUBLICATIONS

list_ciad_date = get_date(list_ciad_links)  
print(len(list_ciad_date))
list_ciad_date[0:3]



34


[None, 'le 8 février à 20:51', '24 décembre 2022']

In [489]:
# CREATION DE VARIABLES DE TEMPS(ANNEE,MOIS,JOUR)

date_list = string_to_date(list_ciad_date)
# list_ciad_date = [ transform_string_date(datte) for datte in list_ciad_date]
list_ciad_year = date_to_year(date_list)
list_ciad_month = date_to_month(date_list)
list_ciad_day = date_to_day(date_list)


In [490]:
date_list

[None,
 datetime.datetime(2023, 2, 8, 20, 51),
 datetime.datetime(2022, 12, 24, 0, 0),
 datetime.datetime(2023, 1, 18, 0, 0),
 datetime.datetime(2017, 2, 23, 8, 37, 0, 708393),
 datetime.datetime(2023, 2, 9, 17, 48),
 datetime.datetime(2021, 2, 23, 8, 37, 0, 712917),
 datetime.datetime(2023, 2, 9, 17, 42),
 datetime.datetime(2023, 1, 24, 21, 25),
 datetime.datetime(2021, 2, 23, 8, 37, 0, 720917),
 datetime.datetime(2023, 1, 28, 8, 28),
 datetime.datetime(2023, 2, 9, 13, 31),
 datetime.datetime(2023, 1, 5, 0, 0),
 datetime.datetime(2023, 1, 20, 0, 0),
 datetime.datetime(2022, 12, 25, 0, 0),
 datetime.datetime(2023, 1, 5, 0, 0),
 datetime.datetime(2022, 12, 30, 0, 0),
 datetime.datetime(2022, 12, 29, 0, 0),
 datetime.datetime(2023, 2, 7, 5, 9),
 datetime.datetime(2023, 1, 3, 0, 0),
 datetime.datetime(2022, 12, 25, 0, 0),
 datetime.datetime(2023, 2, 9, 13, 3),
 datetime.datetime(2023, 1, 25, 18, 57),
 datetime.datetime(2022, 12, 26, 0, 0),
 datetime.datetime(2023, 1, 21, 0, 0),
 datetime.

In [491]:
# IDENTIFICATION DU TYPE DE LOGEMENT
list_ciad_type = AppartDetect(len(list_ciad_texte), list_ciad_texte, list_ciad_title)
list_ciad_type[0:3]


['Maison', 'Appartement', 'Appartement']

In [492]:
# CORRECTION DU PRIX
list_ciad_price = correction_price(list_ciad_price)
list_ciad_price[0:3]

[500000000, 180000000, 60000000]

In [493]:
# OBTENTION DE STANDING
list_ciad_standing = standing(list_ciad_price)
list_ciad_standing[0:3]

['Moyen', 'Economique', 'Economique']

In [494]:
# OBTENTION DU NOMBRE DE PIECES 

list_ciad_room_numb = get_room_numb(list_ciad_texte)
list_ciad_room_numb[0:10]


[4, 2, None, 3, 4, 4, 2, 4, 4, 2]

In [495]:
# CREATION DE DATA FRAME POUR STOCKER LES DONNEES SUR CIAD

dic_ciad = {
'ANNEE':list_ciad_year,
'MOIS':list_ciad_month,
'JOUR':list_ciad_day,
'NB_PIECES':list_ciad_room_numb,
'TYPE':list_ciad_type,
'STANDING':list_ciad_standing,

'COMMUNE':list_ciad_com,
'QUARTIER':list_ciad_quart,
'LOYER':list_ciad_price,
# 'NB_LOYER': NB_LOYER,
'LIEN':list_ciad_links}

# Data_ciad = pd.DataFrame(dic_ciad)
Data_ciad2 = pd.DataFrame(dic_ciad)
# Data_ciad.to_csv('../data/Data_ciad.csv')
# Data_ciad.head(1)

# Data_ciad.to_csv('../data/Data_ciad.csv')
# # pd.read_csv('../data/Data_ciad.csv')
# Data_ciad2.to_csv('../data/Data_ciad2.csv', index = False)
# pd.read_csv('../data/Data_ciad2.csv')

# Data_ciad2.to_csv('../data/Data_ciad4.csv', index = False)
# pd.read_csv('../data/Data_ciad4.csv')
# Data_ciad2.to_csv('../data/Data_ciad5.csv', index = False)
# pd.read_csv('../data/Data_ciad5.csv')
Data_ciad2.to_csv('../data/Data_ciad6.csv', index = False)
pd.read_csv('../data/Data_ciad6.csv')

,ANNEE,MOIS,JOUR,NB_PIECES,TYPE,STANDING,COMMUNE,QUARTIER,LOYER,LIEN
0,NaN,NaN,NaN,4.0,Maison,Moyen,Cocody/Riveira,Ciad,500000000,https://www.facebook.com/groups/15585629877089...
1,2023.0,2.0,8.0,2.0,Appartement,Economique,Cocody/Riveira,Ciad,180000000,https://www.facebook.com/groups/15585629877089...
2,2022.0,12.0,24.0,NaN,Appartement,Economique,Cocody/Riveira,Ciad,60000000,https://www.facebook.com/groups/15585629877089...
3,2023.0,1.0,18.0,3.0,Appartement,Moyen,Cocody/Riveira,Ciad,570000000,https://www.facebook.com/groups/15585629877089...
4,2017.0,2.0,23.0,4.0,Appartement,Haut,Cocody/Riveira,Ciad,700000000,https://www.facebook.com/groups/15585629877089...
5,2023.0,2.0,9.0,4.0,Appartement,Haut,Cocody/Riveira,Ciad,670000000,https://www.facebook.com/groups/15585629877089...
6,2021.0,2.0,23.0,2.0,Appartement,Economique,Cocody/Riveira,Ciad,185000000,https://www.facebook.com/groups/15585629877089...
7,2023.0,2.0,9.0,4.0,Appartement,Haut,Cocody/Riveira,Ciad,670000000,https://www.facebook.com/groups/15585629877089...
8,2023.0,1.0,24.0,4.0,Maison,Economique,Cocody/Riveira,Ciad,80000,https://www.facebook.com/groups/15585629877089...
9,2021.0,2.0,23.0,2.0,Appartement,Economique,Cocody/Riveira,Ciad,180000000,https://www.facebook.com/groups/15585629877089...


<H2>2-MBADON </H2>

In [496]:
# OBTENTION DES PUBLICATION
publication_mbadon = get_all_publications(url_mbadon)
publication_mbadon[0:2]


[<selenium.webdriver.remote.webelement.WebElement (session="b960a1a6-3280-43aa-aaeb-c87bfe828471", element="f30c062b-f5a7-4005-976d-e043722a1c1a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b960a1a6-3280-43aa-aaeb-c87bfe828471", element="b725972b-62ce-4300-bb3b-1ccc2b1d8329")>]

In [497]:
# OBTENTION DES LIENS
list_mbadon_links = get_links(publication_mbadon)
print(len(list_mbadon_links))
list_mbadon_links[0:3]

22


['https://web.facebook.com/groups/1558562987708936/permalink/3538377576394124/?sale_post_id=3538377576394124',
 'https://web.facebook.com/groups/1558562987708936/permalink/3500004476898101/?sale_post_id=3500004476898101',
 'https://web.facebook.com/groups/1558562987708936/permalink/3499773483587867/?sale_post_id=3499773483587867']

In [498]:
# OBTENTION DES TITLES
list_mbadon_title = get_title(publication_mbadon)
len(list_mbadon_title)
list_mbadon_title[1:3]

["COCODY RIVIERA M'BADON TERRAIN DE 2.673 m² AVEC ACD Tél : 07 49 81 65 85",
 "🌹 RIVIERA 4 M'BADON 🌹 Très Belle duplex de 7 pièces à louer dans une zone Résidentielle calme e"]

In [499]:
# OBTENTION DES PRIX
list_mbadon_price = get_price(publication_mbadon)
print(len(list_mbadon_price))
list_mbadon_price[0:3]

22


['350 000 FCFA', '450 000 FCFA', '1 FCFA']

In [500]:
# OBTENTION DES TEXTES

list_mbadon_texte = get_texte(publication_mbadon)

print(len(list_mbadon_texte))
list_mbadon_texte[0:2]

22


['LOCATION APPARTEMENT SKY RIVIERA IV, M’BADON SKY est un appartement T2 avec des pièces aérées et lumineuses au sein d’un immeuble neuf R+4. - 01 séjour lumineux avec grand balcon - 01 cuisine - 01 chambre avec placard de rangement - 01 salle d’eau Commodités: - Chauffe-eau installé - Gardiennage 24h/7j - Parking Interne Loyer: 350.000F CFA &#-#7boudab7363;*- 0709203414',
 "🇨🇮𝐴𝑝𝑝𝑒𝑙𝑒𝑧 𝑒𝑡 𝑒𝑥𝑝𝑟𝑖𝑚𝑒𝑧 𝑣𝑜𝑡𝑟𝑒 𝑏𝑒𝑠𝑜𝑖𝑛 𝑖𝑚𝑚𝑜... 𝗧𝗘𝗥𝗥𝗔𝗜𝗡 𝗘𝗡 𝗩𝗘𝗡𝗧𝗘 : 𝗖𝗢𝗖𝗢𝗗𝗬 𝗥𝗜𝗩𝗜𝗘𝗥𝗔 𝗠'𝗕𝗔𝗗𝗢𝗡 𝗧𝗲́𝗹 ☎ : 𝟬𝟳 𝟰𝟵 𝟴𝟭 𝟲𝟱 𝟴𝟱 (Appel & WhatsApp) 𝑈𝑛𝑒 𝑝𝑎𝑟𝑐𝑒𝑙𝑙𝑒 𝑟𝑒𝑐𝑡𝑎𝑛𝑔𝑢𝑙𝑎𝑖𝑟𝑒 𝑑𝑒 2673 𝑚² 𝑎𝑣𝑒𝑐 𝑢𝑛𝑒 𝑐𝑙𝑜̂𝑡𝑢𝑟𝑒 𝑠𝑢𝑟 𝑙𝑒𝑠 2 𝑙𝑜𝑛𝑔𝑢𝑒𝑢𝑟𝑠 𝑠𝑜𝑢𝑡𝑒𝑛𝑢𝑒 𝑝𝑎𝑟 𝑢𝑛 𝐴𝐶𝐷 𝑠𝑖𝑡𝑢𝑒́ 𝑒𝑛 𝑏𝑜𝑟𝑑𝑢𝑟𝑒 𝑑𝑒 𝑙𝑎 𝑙𝑎𝑔𝑢𝑛𝑒 𝑎𝑣𝑒𝑐 𝑢𝑛 𝑎𝑐𝑐𝑒̀𝑠 𝑏𝑖𝑡𝑢𝑚𝑒́ 𝙋𝙧𝙞𝙭 𝙙𝙚 𝙫𝙚𝙣𝙩𝙚 : 450.000 𝙁 𝘾𝙁𝘼 /𝙢² #terrain #abidjan #cocody #riviera #mbadon"]

In [501]:
# CREATION DE VARIABLES "CONSTANTE"

list_mbadon_com = ["Cocody/Riveira" for i  in range(len(list_mbadon_texte))]
list_mbadon_quart = ["Mbadon" for i  in range(len(list_mbadon_texte))]

len(list_mbadon_com), len(list_mbadon_quart)

(22, 22)

In [506]:
# OBTENTION DES DATES DES PUBLICATIONS

list_mbadon_date = get_date(list_mbadon_links)
print(len(list_mbadon_date))
list_mbadon_date

3 j
4 janvier
3 janvier
le 2 février à 11:00
17 janvier
le 2 février à 22:59
9 janvier
le 5 février à 17:26
12 janvier
8 janvier
le 25 janvier à 23:23
le 27 janvier à 08:39
3 janvier
8 janvier
23 janvier
17 janvier
3 j
3 j
le 5 février à 20:37
4 janvier
le 2 février à 15:28
9 janvier
22


['3 j',
 '4 janvier',
 '3 janvier',
 'le 2 février à 11:00',
 '17 janvier',
 'le 2 février à 22:59',
 '9 janvier',
 'le 5 février à 17:26',
 '12 janvier',
 '8 janvier',
 'le 25 janvier à 23:23',
 'le 27 janvier à 08:39',
 '3 janvier',
 '8 janvier',
 '23 janvier',
 '17 janvier',
 '3 j',
 '3 j',
 'le 5 février à 20:37',
 '4 janvier',
 'le 2 février à 15:28',
 '9 janvier']

In [508]:
# CREATION DE VARIABLES DE TEMPS(ANNEE,MOIS,JOUR)


date_list =string_to_date(list_mbadon_date)

list_mbadon_year =  date_to_year(date_list)
list_mbadon_month = date_to_month(date_list)
list_mbadon_day = date_to_day(date_list)
date_list



[datetime.datetime(2020, 2, 23, 9, 0, 52, 615728),
 datetime.datetime(2023, 1, 4, 0, 0),
 datetime.datetime(2023, 1, 3, 0, 0),
 datetime.datetime(2023, 2, 2, 11, 0),
 datetime.datetime(2023, 1, 17, 0, 0),
 datetime.datetime(2023, 2, 2, 22, 59),
 datetime.datetime(2023, 1, 9, 0, 0),
 datetime.datetime(2023, 2, 5, 17, 26),
 datetime.datetime(2023, 1, 12, 0, 0),
 datetime.datetime(2023, 1, 8, 0, 0),
 datetime.datetime(2023, 1, 25, 23, 23),
 datetime.datetime(2023, 1, 27, 8, 39),
 datetime.datetime(2023, 1, 3, 0, 0),
 datetime.datetime(2023, 1, 8, 0, 0),
 datetime.datetime(2023, 1, 23, 0, 0),
 datetime.datetime(2023, 1, 17, 0, 0),
 datetime.datetime(2020, 2, 23, 9, 0, 52, 660117),
 datetime.datetime(2020, 2, 23, 9, 0, 52, 662118),
 datetime.datetime(2023, 2, 5, 20, 37),
 datetime.datetime(2023, 1, 4, 0, 0),
 datetime.datetime(2023, 2, 2, 15, 28),
 datetime.datetime(2023, 1, 9, 0, 0)]

In [509]:
# IDENTIFICATION DU TYPE DE LOGEMENT
list_mbadon_type = AppartDetect(len(list_mbadon_texte), list_mbadon_texte, list_mbadon_title)
list_mbadon_type[0:4]


['Appartement', 'Maison', 'Maison', 'Appartement']

In [510]:
# CORRECTION DU PRIX
list_mbadon_price = correction_price(list_mbadon_price)
list_mbadon_price[0:3]

[350000000, 450000000, 1000]

In [511]:
# OBTENTION DE STANDING
list_mbadon_standing = standing(list_mbadon_price)
list_mbadon_standing[0:3]

['Moyen', 'Moyen', 'Economique']

In [512]:
# OBTENTION DU NOMBRE DE PIECES 

list_mbadon_room_numb = get_room_numb(list_mbadon_texte)
list_mbadon_room_numb[0:10]


# len(list_mbadon_texte)

[[], None, 7, 4, 3, 4, 2, 4, None, 8]

In [513]:
# CREATION DE DATA FRAME POUR STOCKER LES DONNEES SUR CIAD

dic_mbadon = {
'ANNEE':list_mbadon_year,
'MOIS':list_mbadon_month,
'JOUR':list_mbadon_day,
'NB_PIECES':list_mbadon_room_numb,
'TYPE':list_mbadon_type,
'STANDING':list_mbadon_standing,

'COMMUNE':list_mbadon_com,
'QUARTIER':list_mbadon_quart,
'LOYER':list_mbadon_price,
# 'NB_LOYER': NB_LOYER,
'LIEN':list_mbadon_links}

# Data_mbadon = pd.DataFrame(dic_mbadon)
# Data_mbadon.to_csv('../data/Data_mbadon.csv')
# Data_mbadon.head(1)

# pd.read_csv('../data/Data_mbadon.csv')

# Data_mbadon = pd.DataFrame(dic_mbadon)
# Data_mbadon.to_csv('../data/Data_mbadon2.csv', index = False)
# Data_mbadon.head(1)
# Data_mbadon = pd.DataFrame(dic_mbadon)
# Data_mbadon.to_csv('../data/Data_mbadon3.csv', index = False)
# # Data_mbadon.head(1)
# pd.read_csv('../data/Data_mbadon3.csv')

# Data_mbadon = pd.DataFrame(dic_mbadon)
# Data_mbadon.to_csv('../data/Data_mbadon4.csv', index = False)
# # Data_mbadon.head(1)
# pd.read_csv('../data/Data_mbadon4.csv')


Data_mbadon = pd.DataFrame(dic_mbadon)
Data_mbadon.to_csv('../data/Data_mbadon5.csv', index = False)
# Data_mbadon.head(1)
pd.read_csv('../data/Data_mbadon5.csv')

,ANNEE,MOIS,JOUR,NB_PIECES,TYPE,STANDING,COMMUNE,QUARTIER,LOYER,LIEN
0,2020,2,23,[],Appartement,Moyen,Cocody/Riveira,Mbadon,350000000,https://web.facebook.com/groups/15585629877089...
1,2023,1,4,NaN,Maison,Moyen,Cocody/Riveira,Mbadon,450000000,https://web.facebook.com/groups/15585629877089...
2,2023,1,3,7,Maison,Economique,Cocody/Riveira,Mbadon,1000,https://web.facebook.com/groups/15585629877089...
3,2023,2,2,4,Appartement,Haut,Cocody/Riveira,Mbadon,800000000,https://web.facebook.com/groups/15585629877089...
4,2023,1,17,3,Appartement,Moyen,Cocody/Riveira,Mbadon,450000000,https://web.facebook.com/groups/15585629877089...
5,2023,2,2,4,Maison,Haut,Cocody/Riveira,Mbadon,700000000,https://web.facebook.com/groups/15585629877089...
6,2023,1,9,2,Maison,Economique,Cocody/Riveira,Mbadon,130000000,https://web.facebook.com/groups/15585629877089...
7,2023,2,5,4,Maison,Haut,Cocody/Riveira,Mbadon,700000000,https://web.facebook.com/groups/15585629877089...
8,2023,1,12,NaN,Maison,Haut,Cocody/Riveira,Mbadon,3500000000,https://web.facebook.com/groups/15585629877089...
9,2023,1,8,8,Maison,Haut,Cocody/Riveira,Mbadon,3500000000,https://web.facebook.com/groups/15585629877089...


In [514]:
df1 = pd.read_csv('../data/Data_mbadon2.csv')
df2 = pd.read_csv('../data/Data_mbadon.csv')
df3 = pd.read_csv('../data/Data_ciad.csv')
df4 = pd.read_csv('../data/Data_ciad2.csv')
df5 = pd.read_csv('../data/Data_mbadon3.csv')
df6 = pd.read_csv('../data/Data_ciad3.csv')
df7 = pd.read_csv('../data/Data_mbadon4.csv')
df8 = pd.read_csv('../data/Data_ciad4.csv')
df9 = pd.read_csv('../data/Data_ciad5.csv')
df8 = pd.read_csv('../data/Data_ciad6.csv')
df9 = pd.read_csv('../data/Data_mbadon5.csv')

data_final = pd.concat([df3,df2, df1, df4, df5,df6,df7,df8,df9]).drop_duplicates()
data_final['COMMUNE'] =  'Cocody/Riviera'
data_final.to_csv('../data/Data_final.csv', index=False)
data_final

,ANNEE,MOIS,JOUR,NB_PIECES,TYPE,STANDING,COMMUNE,QUARTIER,LOYER,LIEN
0,2023.0,2.0,9.0,4.0,Maison,Moyen,Cocody/Riviera,Ciad,500000000,https://www.facebook.com/groups/15585629877089...
1,2023.0,2.0,9.0,4.0,Appartement,Haut,Cocody/Riviera,Ciad,670000000,https://www.facebook.com/groups/15585629877089...
2,2023.0,2.0,9.0,4.0,Maison,Moyen,Cocody/Riviera,Ciad,500000000,https://www.facebook.com/groups/15585629877089...
3,2023.0,1.0,10.0,4.0,Appartement,Economique,Cocody/Riviera,Ciad,300000000,https://www.facebook.com/groups/15585629877089...
4,2023.0,2.0,9.0,4.0,Maison,Moyen,Cocody/Riviera,Ciad,500000000,https://www.facebook.com/groups/15585629877089...
...,...,...,...,...,...,...,...,...,...,...
17,2020.0,2.0,23.0,5,Appartement,Haut,Cocody/Riviera,Mbadon,1300000000,https://web.facebook.com/groups/15585629877089...
18,2023.0,2.0,5.0,4,Maison,Moyen,Cocody/Riviera,Mbadon,600000000,https://web.facebook.com/groups/15585629877089...
19,2023.0,1.0,4.0,NaN,Maison,Economique,Cocody/Riviera,Mbadon,4000,https://web.facebook.com/groups/15585629877089...
20,2023.0,2.0,2.0,4,Maison,Haut,Cocody/Riviera,Mbadon,700000000,https://web.facebook.com/groups/15585629877089...


##################################################################################################################################################
##################################################################################################################################################
############################################# [ FIN ] ########################################################
##################################################################################################################################################
##################################################################################################################################################

